In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.10.0 

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import urllib.request
import requests
#from bs4 import BeautifulSoup
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Geocoder to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print("Importing libraries complete!")

Importing libraries complete!


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Table"
table_list = pd.read_html(url)
#print("number of tables in web page = ", len(table_list))
Raw_df = table_list[11] # what we want is the 11th table in the list

Raw_df.head()

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN


In [4]:
Raw_df.drop(columns=['Map', 'Unnamed: 5'], axis=1, inplace = True)
#DataFrame.drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise'
#Raw_df.sort_values("CDN number", axis=0, ascending=True, inplace=True)

In [5]:
Neighbour_df = Raw_df.rename(columns={'CDN number': 'CDN', 
                                     'City-designated neighbourhood':'Neighbourhood', 
                                     'Former city/borough':'Borough',
                                     'Neighbourhoods covered':'NBH_Covered'
                                    })

Neighbour_df.head()

,CDN,Neighbourhood,Borough,NBH_Covered
0,129,Agincourt North,Scarborough,Agincourt and Brimwood
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern
2,20,Alderwood,Etobicoke,Alderwood
3,95,Annex,Old City of Toronto,The Annex and Seaton Village
4,42,Banbury-Don Mills,North York,Don Mills


In [6]:
# Explode/Split Neighbourhood column into multiple rows (removing the hyphen symbol)
new_df = pd.DataFrame(Neighbour_df.Neighbourhood.str.split('-').tolist(), index=Neighbour_df.CDN).stack()
new_df = new_df.reset_index([0, 'CDN'])
new_df['Latitude'] = np.NaN
new_df['Longitude'] = np.NaN
new_df.columns = ['CDN', 'Neighbourhood', 'Latitude', 'Longitude']
new_df.head(10)

,CDN,Neighbourhood,Latitude,Longitude
0,129,Agincourt North,NaN,NaN
1,128,Agincourt South,NaN,NaN
2,128,Malvern West,NaN,NaN
3,20,Alderwood,NaN,NaN
4,95,Annex,NaN,NaN
5,42,Banbury,NaN,NaN
6,42,Don Mills,NaN,NaN
7,34,Bathurst Manor,NaN,NaN
8,76,Bay Street Corridor,NaN,NaN
9,52,Bayview Village,NaN,NaN


In [ ]:
#geolocator = Nominatim(user_agent="Sample_Data")
#location = geolocator.geocode("Humbermede Toronto Ontario")
#print(location.raw)


from functools import partial
#rom geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="Neighbourhood_explorer")

_geocode = partial(geolocator.geocode, language="en")
geocode = lambda query, **kw: _geocode("%s, Toronto Ontario Canada" % query, **kw)

#geocode = lambda query: geolocator.geocode("%s, ON Canada" % query)
#geocode = RateLimiter(geocode_, min_delay_seconds=1)

def eval_lat(x):
    try:
        #print(x)
        return (x.latitude)
    except:
        return (None)
def eval_lon(x):
    try:
        return (x.longitude)
    except:
        return (None)

    
new_df['Latitude'] = new_df['Neighbourhood'].apply(geocode).apply(lambda x: eval_lat(x))
new_df['Longitude'] = new_df['Neighbourhood'].apply(geocode).apply(lambda x: eval_lon(x))

print("Geo Coding complete!")



Geo Coding complete!


In [ ]:
#new_df.to_csv('TorontoNeighbourhoodGeocoding.csv', index = False)

In [7]:
#Import the geocode data from csv file 
new_df = pd.read_csv("TorontoNeighbourhoodGeocoding.csv")


In [8]:
new_df

,CDN,Neighbourhood,Latitude,Longitude
0,129,Agincourt North,43.808038,-79.266439
1,128,Agincourt South,43.785353,-79.278549
2,128,Malvern West,43.790679,-79.248491
3,20,Alderwood,43.601717,-79.545232
4,95,Annex,43.670338,-79.407117
5,42,Banbury,43.742796,-79.369957
6,42,Don Mills,43.775347,-79.345944
7,34,Bathurst Manor,43.665519,-79.411937
8,76,Bay Street Corridor,43.667342,-79.388457
9,52,Bayview Village,43.769197,-79.376662


In [9]:
df1 = new_df[new_df.isna().any(axis=1)]
df1

,CDN,Neighbourhood,Latitude,Longitude
17,122,Birchcliffe,NaN,NaN
40,33,Clanton Park,NaN,NaN
56,26,CFB,NaN,NaN
88,22,Humbermede,NaN,NaN
93,14,City Centre West,NaN,NaN
104,114,Lambton Baby Point,NaN,NaN
143,67,Playter Estates,NaN,NaN
172,3,Beaumond Heights,NaN,NaN
182,1,Clairville,NaN,NaN
187,91,Pellam Park,NaN,NaN


In [10]:
new_df.dropna(axis=0, inplace=True)
print("shape before removing duplicates: ",new_df.shape)
new_df.drop_duplicates(subset ="CDN", keep = 'first', inplace = True) 
print("shape after removing duplicates: ",new_df.shape)

shape before removing duplicates:  (195, 4)
shape after removing duplicates:  (136, 4)


In [11]:

address = 'Casa Loma,ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Casa Loma,ON are 43.6781015, -79.409415775.


In [12]:
# create map of Toronto using latitude and longitude values created above
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [13]:
Toronto_df = pd.merge(Neighbour_df, new_df, how = 'inner', on = "CDN")
Toronto_df.head(10)

,CDN,Neighbourhood_x,Borough,NBH_Covered,Neighbourhood_y,Latitude,Longitude
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,Agincourt North,43.808038,-79.266439
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,Agincourt South,43.785353,-79.278549
2,20,Alderwood,Etobicoke,Alderwood,Alderwood,43.601717,-79.545232
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,Annex,43.670338,-79.407117
4,42,Banbury-Don Mills,North York,Don Mills,Banbury,43.742796,-79.369957
5,34,Bathurst Manor,North York,Bathurst Manor,Bathurst Manor,43.665519,-79.411937
6,76,Bay Street Corridor,Old City of Toronto,"Bay Street, Financial District",Bay Street Corridor,43.667342,-79.388457
7,52,Bayview Village,North York,Bayview Village,Bayview Village,43.769197,-79.376662
8,49,Bayview Woods-Steeles,North York,Bayview Woods,Bayview Woods,43.798127,-79.382973
9,39,Bedford Park-Nortown,North York,"Bedford Park, Ledbury Park, and Nortown",Bedford Park,43.737388,-79.410925
